In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils


from google.colab import drive
drive.mount('/content/drive')

# Path to the Shakespeare text file on Google Drive
file_path = '/content/drive/My Drive/shakespeare.txt'

# Reading the text file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read().splitlines()
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Initialize and fit tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1

# Create sequences
input_sequence = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)

# Padding sequences to ensure all are the same length
input_sequence = pad_sequences(input_sequence, padding='pre')
max_len = input_sequence.shape[1]  # Correctly calculating max_len here
vocab_size = len(tokenizer.word_index) + 1  # Include +1 for zero padding

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional

def build_generator(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=max_len-1),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        LSTM(128),
        Dense(vocab_size, activation='softmax')
    ])
    return model

def build_discriminator(vocab_size, max_len):
    model = Sequential([
        Embedding(vocab_size, 100, input_length=max_len),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        LSTM(128),
        Dense(1, activation='sigmoid')
    ])
    return model


# Now use these variables to build and compile your models
generator = build_generator(vocab_size, max_len)
discriminator = build_discriminator(vocab_size, max_len)

from tensorflow.keras.optimizers import Adam

# Compile the discriminator
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Setup and compile the GAN
discriminator.trainable = False
gan_input = Input(shape=(max_len-1,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
# GAN Model where the discriminator's weights are frozen during generator training
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False
gan_input = Input(shape=(max_len-1,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy')

def train_gan(epochs, batch_size):
    for epoch in range(epochs):
        idx = np.random.randint(0, input_sequence.shape[0], batch_size)
        real_seqs = input_sequence[idx]

        noise = np.random.normal(0, 1, (batch_size, max_len-1))
        fake_seqs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_seqs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_seqs, np.zeros((batch_size, 1)))

        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        print(f"Epoch {epoch+1}/{epochs}, Discriminator Loss: {(d_loss_real + d_loss_fake) / 2}, Generator Loss: {g_loss}")


def generate_text(seed_text, next_words=100):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted_probs = generator.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        predicted_word = tokenizer.index_word.get(predicted_index, 'unknown')
        seed_text += " " + predicted_word
    return seed_text

# Example seed text
generated_text = generate_text("adorned in shades of red")
print(generated_text)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


adorned in shades of red sail sail windy windy windy windy bonds after forsake forsake believe let habitation rain siren worthier surety vermilion vermilion coward coward spread might party party party party snow divine perfect'st perfect'st wrong perfect'st said correspondence life life slow crossed grossly enough ending siege siege bones receipt receipt receipt mind commence men's men's men's men's men's blow blow blow blow bonds bonds wombs refigured spoil spoil spoil spoil dwells interim whom thrust whom curls curls winds felt as different plead plead semblance ending blindness valley men's men's men's men's blow blow blow blow bonds bonds wombs refigured spoil spoil spoil spoil


In [ ]:
import tensorflow_probability as tfp

In [ ]:
#### prev
def generate_text(seed_text, next_words=100):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted_probs = generator.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        predicted_word = tokenizer.index_word.get(predicted_index, 'unknown')
        seed_text += " " + predicted_word
    return seed_text

# Example seed text
generated_text = generate_text("adorned in shades of red")
print(generated_text)


adorned in shades of red sail sail windy windy windy windy bonds after forsake forsake believe let habitation rain siren worthier surety vermilion vermilion coward coward spread might party party party party snow divine perfect'st perfect'st wrong perfect'st said correspondence life life slow crossed grossly enough ending siege siege bones receipt receipt receipt mind commence men's men's men's men's men's blow blow blow blow bonds bonds wombs refigured spoil spoil spoil spoil dwells interim whom thrust whom curls curls winds felt as different plead plead semblance ending blindness valley men's men's men's men's blow blow blow blow bonds bonds wombs refigured spoil spoil spoil spoil


In [ ]:
#EXP

import numpy as np

def sample(preds, temperature=1.0):
    """ Helper function to sample an index from a probability array. """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Adding a small number to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(seed_text, next_words=100, temperature=0.8):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted_probs = generator.predict(token_list, verbose=0)[0]
        next_index = sample(predicted_probs, temperature)  # Use temperature
        next_word = tokenizer.index_word.get(next_index, 'unknown')
        seed_text += " " + next_word
    return seed_text

# Generate text with a bit of randomness
generated_text = generate_text("Adorned in shades of red", next_words=50, temperature=0.8)
print(generated_text)


Adorned in shades of red presenteth lesson faults rocks chips affairs impiety concealed charter indeed deserts death's parallels adding rolling lover dial's wooing kills measure unfolding heinous divide dressing receiving relief monument dwells soundless dumb bore watching straying swart burn substance spacious temperate convertest answered mad rest ransoms children's vexed possessing acceptable praising costs apparel


In [ ]:
import random

def generate_sonnet_line(next_words=10):
    # Start with a random initial word from the vocabulary
    initial_word = random.choice(list(tokenizer.word_index.keys()))
    line = generate_text(initial_word, next_words=next_words)  # Generate the rest of the line
    return line

def create_sonnet():
    # Generate lines for each rhyme scheme
    a1 = generate_sonnet_line()
    a2 = generate_sonnet_line()
    b1 = generate_sonnet_line()
    b2 = generate_sonnet_line()
    c1 = generate_sonnet_line()
    c2 = generate_sonnet_line()
    d1 = generate_sonnet_line()
    d2 = generate_sonnet_line()
    e1 = generate_sonnet_line()
    e2 = generate_sonnet_line()
    f1 = generate_sonnet_line()
    f2 = generate_sonnet_line()
    g1 = generate_sonnet_line()
    g2 = generate_sonnet_line()

    # Concatenate lines according to rhyme scheme
    sonnet_structure = "\n".join([
        a1, b1, a2, b2,  # ABAB
        c1, d1, c2, d2,  # CDCD
        e1, f1, e2, f2,  # EFEF
        g1, g2  # GG
    ])
    return sonnet_structure

# Example usage to generate a sonnet
generated_sonnet = create_sonnet()
print(generated_sonnet)


credit several task forbear acknowledge dare single siege methinks vex is
contents composition linger bare ransom defendant hill bark any proudest greatest
woe impute buy cry possesseth lascivious lesser churls trees bemoaned faster
stern eclipses unstained affords stage jealousy nativity shakespeare ward gav'st votary
call foe legacy unto resting straying receiv'st seal skill statues fate
chronicle deformed'st neglected seasoned defeat gems several governs single feed'st impregnable
termed costly clerk limbecks arise jade bids now head bare mow
jewels corrupting wish physic dyed separable sets pricked cruel worthier worthier
can deep graced onset presage as fed parts sin interim durst
through repay together wherefore presents wife due paying drink determinate unlettered
candles sole few already soul receiv'st goodness greet perish sovereign steepy
bail slumbers deface outworn travel describe forwards dressed day outworn see
curls children's iniquity graves perhaps accident breathe wea